Document showing how to consolidate the possible subsegments into a major trend. 

# Dependencies

In [1]:
from oanda_api import OandaAPI
from candlelist import CandleList

import os
try:
    user_paths = os.environ['PYTHONPATH'].split(os.pathsep)
    print(user_paths)
except KeyError:
    user_paths = []
    

['/Users/ernesto/PycharmProjects/FOREX/src']


# Test case

Here I am going to use the uptrend shown below:
that goes from EUR_NZD (D) 26MAR2019 to 25JUL2019
![alt text](img/Consolidate_trend.png)

Let's get a CandleList object for this uptrend

In [2]:
oanda = OandaAPI(url='https://api-fxtrade.oanda.com/v1/candles?',
                     instrument='EUR_NZD',
                     granularity='D',
                     alignmentTimezone='Europe/London',
                     dailyAlignment=22)

oanda.run(start='2019-03-26T22:00:00',
              end='2019-07-25T22:00:00',
              roll=True)

candle_list = oanda.fetch_candleset()

cl = CandleList(candle_list, instrument='EUR_NZD', type='long')

Now, let's get a PivotList object from this CandleList

In [3]:
pl = cl.get_pivotlist(outfile='img/cl_0_02_a.png', th_up=0.02, th_down=-0.02)

![alt text](img/cl_0_02_a.png?1)

`th_up` and `th_down` will control the sensitivity of the pivot points. So if we make it lower we will increase the number of pivots

In [4]:
pl_001 = cl.get_pivotlist(outfile='img/cl_0_01_a.png', th_up=0.01, th_down=-0.01)

![alt text](img/cl_0_01_a.png)

We can get a SegmentList object from `pl`

In [5]:
slist = pl_001.slist

And then we can use the function `merge_segments()` to identify the principal segment ignoring the retracements

In [6]:
mslist=slist.merge_segments(min_n_candles=5, diff_in_pips=200, outfile="img/mcl_0_01_1.png")

/Users/ernesto/PycharmProjects/FOREX/src/segmentlist.py:165: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  plt.scatter(xarr[[ix_start]], yarr[[ix_start]], color='g')
/Users/ernesto/PycharmProjects/FOREX/src/segmentlist.py:166: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  plt.scatter(xarr[[ix_end]], yarr[[ix_end]], color='r')
/Users/ernesto/PycharmProjects/FOREX/src/segmentlist.py:171: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will

![alt text](img/mcl_0_01_1.png)

If we want to consolidate the downtrend retracement into the major uptrend we need to increase either `min_n_candles` or `diff_in_pips` in order to make it harder the downtrend retracement to be considered a different trend and be considered a retracement merged into the major uptrend

In [7]:
mslist=slist.merge_segments(min_n_candles=20, diff_in_pips=200, outfile="img/mcl_0_01_ncandles20b.png")

![alt text](img/mcl_0_01_ncandles20b.png)

Let's try now with a more complex oanda query composed of 2 major trends

In [9]:
oanda.run(start='2014-11-26T22:00:00',
              end='2015-09-21T22:00:00',
              roll=True)

candle_list2 = oanda.fetch_candleset()

cl2 = CandleList(candle_list2, instrument='EUR_NZD', type='long')

![alt text](img/Consolidate_trend1.png)

Let's get the pivot points again , using the 0.02 as `th_up` and `th_down` value

In [10]:
pl2 = cl2.get_pivotlist(outfile='img/cl2_0_02_b.png', th_up=0.02, th_down=-0.02)

![alt text](img/cl2_0_02_a.png)

We see a lot of segments that need to be merged in a downtrend and an uptrend. For this, we use again the `merge_segments` function

In [12]:
slist2 = pl2.slist

In [14]:
mslist2=slist2.merge_segments(min_n_candles=20, diff_in_pips=100000, outfile="img/mcl2_0_01_ncandles20e.png")

![alt text](img/mcl2_0_01_ncandles20e.png)